## Setup

In [ ]:
! git clone https://github.com/tloen/alpaca-lora.git
! cd alpaca-lora; pip install -r requirements.txt

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 493, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 493 (delta 11), reused 15 (delta 7), pack-reused 473
Receiving objects: 100% (493/493), 13.85 MiB | 17.68 MiB/s, done.
Resolving deltas: 100% (296/296), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-ybog6z6y
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-ybog6z6y
  Resolved https://github.com/huggingface/peft.git to commit df71b84341ae1ab3bc9b0d5f906d7a524850b63b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-13gqarvv
  Running command git clone --fil

In [ ]:
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00


In [ ]:
! export BASE_MODEL=decapoda-research/llama-7b-hf

## Structure Data

In [ ]:
# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#See https://drive.google.com/file/d/16JuBgYFkgro6tPzArrOA9JN0YxiqGBAc/view?usp=share_link
file_id = '16JuBgYFkgro6tPzArrOA9JN0YxiqGBAc'

download = drive.CreateFile({'id': file_id})

# Download the file to a local disc
download.GetContentFile('data1000_v1.json')

### Restructure

In [ ]:
def clean_text(text):
    text = text.replace('{', '').replace('}', '').replace('[', '').replace(']', '')
    text = text.replace('\"', '').replace(',', ', ').replace(':', ': ')
    text = ' '.join(text.split()).strip()  # Remove extra spaces
    return text

In [ ]:
import json

# Read the original JSON file
with open('data1000_v1.json', 'r') as infile:
    data = json.load(infile)

# Process and restructure the data
reformatted_data = []

for item in data:
  response_text = f"{item['output_reaction_inputs']} {item['output_reaction_conditions']}"
  #response_text = clean_text(response_text)
  new_item = {
        'instruction': item['input_text'],
        'input': '',
        'output': response_text
        }
  reformatted_data.append(new_item)

# Write the reformatted data to a new JSON file
with open('cleaned_data1000_v1.json', 'w') as outfile:
    json.dump(reformatted_data, outfile, indent=2)


## Fine-tuning

In [ ]:
! python alpaca-lora/finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path './cleaned_data1000_v1.json' \
    --output_dir './fine-tuned'\
    --val_set_size 200


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/

In [ ]:
!pip install ngrok

In [ ]:
! python alpaca-lora/generate.py \
    --load_8bit \
    --base_model 'decapoda-research/llama-7b-hf' \
    --lora_weights 'fine-tuned'

In [ ]:
!ngrok http 5000